<a href="https://colab.research.google.com/github/Yash-Yelave/Recomendation_systems/blob/main/RS_pr4_arl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip uninstall mlxtend


Found existing installation: mlxtend 0.23.4
Uninstalling mlxtend-0.23.4:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/mlxtend-0.23.4.dist-info/*
    /usr/local/lib/python3.11/dist-packages/mlxtend/*
Proceed (Y/n)? y
  Successfully uninstalled mlxtend-0.23.4


In [ ]:
pip install mlxtend


  Using cached mlxtend-0.23.4-py3-none-any.whl.metadata (7.3 kB)
Using cached mlxtend-0.23.4-py3-none-any.whl (1.4 MB)


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
###################
# Data Preprocessing
###################

import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 500)
pd.set_option("display.expand_frame_repr", False)

df_original = pd.read_excel("/content/drive/MyDrive/Engineering/Sem5/RS/dataset/online_retail_II.xlsx",sheet_name = ["Year 2009-2010", "Year 2010-2011"])

In [ ]:
# Combine two DataFrames for the specified years
df1 = df_original["Year 2009-2010"]
df2 = df_original["Year 2010-2011"]

# Use pd.concat instead of append
df_ = pd.concat([df1, df2])

# Copy the concatenated DataFrame
df = df_.copy()

# Ensure 'InvoiceDate' is a valid datetime column
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')

# Remove rows where 'InvoiceDate' could not be parsed
df = df.dropna(subset=['InvoiceDate'])

# Filter out rows with negative values in numeric columns ('Quantity', 'Price')
df = df[(df['Quantity'] >= 0) & (df['Price'] >= 0)]

# Drop rows with missing 'Customer ID' if necessary
df = df.dropna(subset=['Customer ID'])

# Inspect the cleaned DataFrame
print("Shape of cleaned DataFrame:", df.shape)
print("Dataframe description (after cleaning):")
print(df.describe().T)

# Re-inspect the DataFrame after handling negative or missing values
print("Cleaned data description:")
print(df.describe().T)


Shape of cleaned DataFrame: (805620, 8)
Dataframe description (after cleaning):
                count                           mean                  min                  25%                  50%                  75%                  max          std
Quantity     805620.0                      13.307665                  1.0                  2.0                  5.0                 12.0              80995.0   144.306739
InvoiceDate    805620  2011-01-02 10:26:35.704326144  2009-12-01 07:45:00  2010-07-07 12:08:00  2010-12-03 15:10:00  2011-07-28 13:28:00  2011-12-09 12:50:00          NaN
Price        805620.0                       3.206279                  0.0                 1.25                 1.95                 3.75              10953.5    29.197901
Customer ID  805620.0                    15331.85625              12346.0              13982.0              15271.0              16805.0              18287.0  1696.768395
Cleaned data description:
                count                  

In [ ]:
def retail_data_prep(dataframe):
    # Drop missing values
    dataframe.dropna(inplace=True)

    # Ensure the "Invoice" column is treated as strings
    dataframe["Invoice"] = dataframe["Invoice"].astype(str)

    # Filter out rows with "C" in the "Invoice" column
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]

    # Keep only rows with positive Quantity and Price
    dataframe = dataframe[dataframe["Quantity"] > 0]
    dataframe = dataframe[dataframe["Price"] > 0]

    return dataframe


In [ ]:
df.describe().T
# as you can see negative values are no longer exist

df.isnull().sum()
# no more missing values

# limitation for outliers
def outlier_threshold(dataframe, varibale):
    quartile1 = dataframe[varibale].quantile(0.01)
    quartile3 = dataframe[varibale].quantile(0.99)
    # the reason why 0.01 and 0.99 are used is arrange outliers without making harsh changes in dataset
    interquartile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquartile_range
    low_limit = quartile1 - 1.5 * interquartile_range
    return low_limit, up_limit

# suppress outliers
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_threshold(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")

In [ ]:


###################
# Prepare ARL Data Structure
###################

# Invoice-Product Matrix:

# Description   NINE DRAWER OFFICE TIDY   SET 2 TEA TOWELS I LOVE LONDON    SPACEBOY BABY GIFT SET
# Invoice
# 536370                              0                                 1                       0
# 536852                              1                                 0                       1
# 536974                              0                                 0                       0
# 537065                              1                                 0                       0
# 537463                              0                                 0                       1

# reduction of the dataset to a single country
df_fr = df[df["Country"] == "France"]
df_fr.head()

df_fr.shape

df_fr.groupby(["Invoice", "StockCode"]).agg({"Quantity": "sum"}). \
    unstack(). \
    fillna(0). \
    applymap(lambda x: 1 if x > 0 else 0).iloc[0:8, 0:8]



Quantity                                          
StockCode    10002 10120 10125 10135 11001 15036 15039 16012
Invoice                                                     
489439           0     0     0     0     0     0     0     0
489557           0     0     0     0     0     0     0     0
489883           0     0     0     0     0     0     0     0
490139           0     0     0     0     0     0     0     0
490152           0     0     0     0     0     0     0     0
490458           1     0     0     0     0     0     0     0
490684           0     0     0     0     0     0     0     0
490959           1     0     0     0     0     0     0     0

In [ ]:

def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['Invoice', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)

fr_inv_pro_df = create_invoice_product_df(df_fr, id=True)

In [ ]:

# Having product names as variable names causes it to take up a lot of memory and the code to run slowly,
# so it is healthier to name the variables with their stockCodes, not the product names.

fr_inv_pro_df.iloc[0:8, 0:8]

# reach description via stock code
def check_id(dataframe, stock_code):
    product_name = dataframe[dataframe["StockCode"] == stock_code][["Description"]].values[0].tolist()
    print(product_name)

check_id(df_fr, 10002)

['INFLATABLE POLITICAL GLOBE ']


In [ ]:
from mlxtend.frequent_patterns import association_rules


In [ ]:
pip install -U mlxtend


In [ ]:
import mlxtend
print(mlxtend.__version__)

0.23.4


In [ ]:
help(association_rules)


Help on function association_rules in module mlxtend.frequent_patterns.association_rules:

association_rules(df: pandas.core.frame.DataFrame, num_itemsets: Optional[int] = 1, df_orig: Optional[pandas.core.frame.DataFrame] = None, null_values=False, metric='confidence', min_threshold=0.8, support_only=False, return_metrics: list = ['antecedent support', 'consequent support', 'support', 'confidence', 'lift', 'representativity', 'leverage', 'conviction', 'zhangs_metric', 'jaccard', 'certainty', 'kulczynski']) -> pandas.core.frame.DataFrame
    Generates a DataFrame of association rules including the
    metrics 'score', 'confidence', and 'lift'
    
    Parameters
    -----------
    df : pandas DataFrame
      pandas DataFrame of frequent itemsets
      with columns ['support', 'itemsets']
    
    df_orig : pandas DataFrame (default: None)
      DataFrame with original input data. Only provided when null_values exist
    
    num_itemsets : int (default: 1)
      Number of transactions 

In [ ]:
from mlxtend.frequent_patterns import association_rules


In [ ]:
# items that occur frequently together and reach a predefined level of support and confidence
frequent_itemsets = apriori(fr_inv_pro_df,
                            min_support=0.01,
                            use_colnames=True)

# association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)

In [ ]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)


In [ ]:


#################
# Association Rules
#################

# items that occur frequently together and reach a predefined level of support and confidence
frequent_itemsets = apriori(fr_inv_pro_df,
                            min_support=0.01,
                            use_colnames=True)

frequent_itemsets.sort_values("support", ascending=False)

# association rules
rules = association_rules(frequent_itemsets,
                          metric="support",
                          min_threshold=0.01)



In [ ]:

#filtering associatiion fules with support, confidence and lift values
rules[(rules["support"] > 0.05)
      & (rules["confidence"] > 0.1)
      & (rules["lift"] > 5)].sort_values("confidence", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
47335,"(21080, 21086, POST)",(21094),0.078176,0.127036,0.074919,0.958333,7.543803,1.0,0.064987,20.951140,0.941005,0.575000,0.952270,0.774038
14590,"(21080, 21086)",(21094),0.096091,0.127036,0.091205,0.949153,7.471534,1.0,0.078998,17.168295,0.958237,0.691358,0.941753,0.833551
14591,"(21080, 21094)",(21086),0.096091,0.138436,0.091205,0.949153,6.856231,1.0,0.077903,16.944083,0.944949,0.636364,0.940982,0.803988
47336,"(21080, 21094, POST)",(21086),0.079805,0.138436,0.074919,0.938776,6.781273,1.0,0.063871,14.072204,0.926472,0.522727,0.928938,0.739976
1585,(21094),(21086),0.127036,0.138436,0.115635,0.910256,6.575264,1.0,0.098049,9.600279,0.971305,0.771739,0.895836,0.872775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5092,(22629),(22631),0.130293,0.087948,0.060261,0.462500,5.258796,1.0,0.048802,1.696841,0.931167,0.381443,0.410670,0.573843
576,(20724),(22356),0.136808,0.084691,0.058632,0.428571,5.060440,1.0,0.047046,1.601792,0.929560,0.360000,0.375699,0.560440
30421,(22629),"(22630, POST)",0.130293,0.073290,0.055375,0.425000,5.798889,1.0,0.045825,1.611670,0.951531,0.373626,0.379525,0.590278
30470,(22629),"(22631, POST)",0.130293,0.071661,0.050489,0.387500,5.407386,1.0,0.041152,1.515655,0.937175,0.333333,0.340219,0.546023


In [ ]:

##################
# Product Recommendation
##################

def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

    return recommendation_list[0:rec_count]

arl_recommender(rules, 22492, 2)

[21914, 21080]

In [ ]:

# checking the product names from the id of the products going to recommend

def check_id(dataframe, stock_code):
    product_names = []
    for i in stock_code:
        product_name = dataframe[dataframe["StockCode"] == i][["Description"]].values[0].tolist()
        print(f"{i} : {product_name}")


check_id(df_fr, arl_recommender(rules, 22492, 2))


21914 : ['BLUE HARMONICA IN BOX ']
21080 : ['SET/20 RED SPOTTY PAPER NAPKINS ']


In [ ]:
#################
# Script
#################

def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [ ]:
def retail_data_prep(dataframe):
    dataframe.dropna(inplace=True)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[dataframe["Quantity"] > 0]
    dataframe = dataframe[dataframe["Price"] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    return dataframe


def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['Invoice', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)



def create_rules(dataframe, id=True, country="France"):
    dataframe = dataframe[dataframe['Country'] == country]  # ülkeye göre veriyi indirge
    dataframe = create_invoice_product_df(dataframe, id)
    frequent_itemsets = apriori(dataframe, min_support=0.01, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
    return rules


df = df_.copy()

df = retail_data_prep(df)
rules = create_rules(df)

rules[(rules["support"] > 0.05)
      & (rules["confidence"] > 0.1)
      & (rules["lift"] > 5)].sort_values("confidence", ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
47335,"(21080, 21086, POST)",(21094),0.078176,0.127036,0.074919,0.958333,7.543803,1.0,0.064987,20.951140,0.941005,0.575000,0.952270,0.774038
14590,"(21080, 21086)",(21094),0.096091,0.127036,0.091205,0.949153,7.471534,1.0,0.078998,17.168295,0.958237,0.691358,0.941753,0.833551
14591,"(21080, 21094)",(21086),0.096091,0.138436,0.091205,0.949153,6.856231,1.0,0.077903,16.944083,0.944949,0.636364,0.940982,0.803988
47336,"(21080, 21094, POST)",(21086),0.079805,0.138436,0.074919,0.938776,6.781273,1.0,0.063871,14.072204,0.926472,0.522727,0.928938,0.739976
1585,(21094),(21086),0.127036,0.138436,0.115635,0.910256,6.575264,1.0,0.098049,9.600279,0.971305,0.771739,0.895836,0.872775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5092,(22629),(22631),0.130293,0.087948,0.060261,0.462500,5.258796,1.0,0.048802,1.696841,0.931167,0.381443,0.410670,0.573843
576,(20724),(22356),0.136808,0.084691,0.058632,0.428571,5.060440,1.0,0.047046,1.601792,0.929560,0.360000,0.375699,0.560440
30421,(22629),"(22630, POST)",0.130293,0.073290,0.055375,0.425000,5.798889,1.0,0.045825,1.611670,0.951531,0.373626,0.379525,0.590278
30470,(22629),"(22631, POST)",0.130293,0.071661,0.050489,0.387500,5.407386,1.0,0.041152,1.515655,0.937175,0.333333,0.340219,0.546023


In [ ]:
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

    return recommendation_list[0:rec_count]


def check_id(dataframe, stock_code):
    product_names = []
    for i in stock_code:
        product_name = dataframe[dataframe["StockCode"] == i][["Description"]].values[0].tolist()
        print(f"{i} : {product_name}")

check_id(df_fr, arl_recommender(rules, 22492, 2))

21914 : ['BLUE HARMONICA IN BOX ']
21080 : ['SET/20 RED SPOTTY PAPER NAPKINS ']


**Note:** This cell originally produced a `NameError` because `frequent_itemsets` was not defined. A subsequent cell (`o527bxB3C8cy`) correctly defines and uses `frequent_itemsets` to create the `rules` DataFrame, making this cell redundant.